In this notebook, we are going to model a transformer(Time Series Transformer). Unlike the LSTMs, they don't understand "Order" naturally and we must inject positional encodings so the model knows the sequence order.

In [ ]:
#The Setup(Data and positional encoding)

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import math
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import os

# 1. Load & Scale Data (Same as LSTM)
PROCESSED_PATH = os.path.join("..", "data", "processed")
df = pd.read_csv(os.path.join(PROCESSED_PATH, "daily_sales_sentiment.csv"))
df['ds'] = pd.to_datetime(df['ds'])

# Features: Sales, Sentiment, Volume
data = df[['y', 'sentiment_avg', 'sales_volume']].values
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# 2. Sequence Creation Function
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length, 0]) 
    return np.array(X), np.array(y)

SEQ_LENGTH = 30
X, y = create_sequences(scaled_data, SEQ_LENGTH)

# Split
train_size = int(len(X) * 0.8)
X_train = torch.FloatTensor(X[:train_size])
y_train = torch.FloatTensor(y[:train_size]).view(-1, 1)
X_test = torch.FloatTensor(X[train_size:])
y_test = torch.FloatTensor(y[train_size:]).view(-1, 1)

# 3. Positional Encoding (The "Transformer" Secret Sauce)
# This class adds a mathematical pattern to the data so the model knows "Time"
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        return x + self.pe[:x.size(0), :]